<p style="font-weight:bold;"> <span style="font-size: 36px"> Financial Performance Specifications </span> </p>

The aim of this notebook is to document the Financial Performance reporting system. The process is pictorially explained in the following flowchart. 

<a name="TOC"></a>
<center><img src="../Images/SpecificationsPL.PNG" width="45%" style="float:center;">

Throughout this notebook we adopt the following variable notation:  we use the words *computed* vs. *expected* to differentiate the results provided by the application vs. this test notebook, respectively.

In the following the Dimensions, Parameters, Nominal Cashflows, and Actuals are imported from the corresponding csv files in <code>Initialization/Systemorph/</code>

In [1]:
#!eval-notebook "../Initialization/InitSystemorphToMemory"

---

# Process Data

<a id='year-and-period'></a>
## Current Period, Reporting Node and Currency Type

Please select the target period (i.e. year and month) and Reporting Node based on the imported data:

In [1]:
var year          = 2021                    ;
var month         = 3                       ;
var reportingNode = "CH"                    ;

and the type of Currency among <code>Contractual</code>, <code>Functional</code>, and <code>Group</code>

In [1]:
var currencyType  = CurrencyType.Contractual ;

Import the Report definitions, initialize the Workspace and the Test Suite:

In [1]:
#!import "../Report/ReportScopes"
#!import "../Report/ReportConfigurationAndUtils"

In [1]:
Workspace.InitializeFrom(DataSource);

In [1]:
var period        = (year, month)                                                        ;
var reportStorage = new ReportStorage(Workspace, Report)                                 ;
var Test          = Scopes.ForSingleton().WithStorage(reportStorage).ToScope<IUniverse>();

In [1]:
await reportStorage.InitializeReportIndependentCacheAsync();
await reportStorage.InitializeAsync(period, reportingNode, null, currencyType);

## View list of Identities

Based on the Cashflows imported, the list of all possible identities, i.e. combinations of valid Contractual and Functional Currencies, Liability Types, (Re)Insurance and Oci, is contructed and reported here below:

In [1]:
var identities = reportStorage.GetIdentities(period, reportingNode, null, currencyType);

In [1]:
identities

<br/><br/>
<a id='report'></a>
# Report

Please select here below the target Data Node for executing the tests below:

In [1]:
var dataNode = "DT1.1";

The Group of Contract selected is:

In [1]:
var groupOfContract = (await Workspace.Query<GroupOfContract>().Where(x => x.SystemName == dataNode).ToArrayAsync()).FirstOrDefault();
groupOfContract

where 
- the **OciType** can be <code>Default</code> for contracts with non-zero OCI, and <code>null</code> otherwise, 
- the **Liability Type** can be <code>LRC</code> for Liability for Remaining Coverage, and <code>LIC</code> for Liability of Incurred Claims,
- the **Partner** can be <code>null</code> for Insurance contracts, or equal to the Partner System Name for Reinsurance contracts.

The summary of the Financial Performance report is shown below

In [1]:
Report.ForDataCube( Test.GetScopes<FinancialPerformance>(identities).Aggregate().FinancialPerformance.Filter(("GroupOfContract", dataNode)) )
      .WithQuerySource(DataSource)
      .SliceRowsBy("VariableType", "EstimateType")
      .SliceColumnsBy(CurrencyGrouper(currencyType), "LiabilityType","GroupOfContract")
      .ReportGridOptions(reportHeight: 900, headerColumnWidth: 500, groupDefaultExpanded: 3)
      .ToReport() with { Height = 600 }

<br/><br/>
<a name='inspection'></a>
# Inspection

## Fulfillment Cashflows

The Fulfillment Cashflow (FCF) corresponds to the sum of the Best Estimate [Present Value](../Import/ImportScopeCalculation#current-and-locked) (PV) and the [Risk Adjustment](../Import/ImportScopeCalculation#risk-adjustment) (RA), both summed over Amounty Types, and both discounted with the Locked-In curve in the BBA valuation approach:

$$
\text{FCF}(\text{AoC}) =  \text{PV Locked}(\text{AoC})
\bigg|_{\substack{\text{Non Attributable} \\ \text{Amount Types} \\ \text{excluded}}}  
+ \text{RA Locked}(\text{AoC}) ~.
$$

### Non Financial

The computed value for the non financial contribution to the FCF is

In [1]:
var computedDeltaFCF_nonFinancial = Test.GetScopes<FcfChangeInEstimate>(identities).Aggregate()
                                        .FpNonFinancial.Filter(("GroupOfContract", dataNode)).Aggregate().Value;

In [1]:
computedDeltaFCF_nonFinancial

This value can be independently computed starting from the full list of FCFs

In [1]:
var FCFs = Test.GetScopes<Fcf>(identities).Aggregate().Fcf.Filter(("GroupOfContract", dataNode));

In [1]:
var deltaFCF = FCFs.Filter(("VariableType", "!BOP"),("VariableType", "!EOP")) +
               FCFs.Filter(("VariableType", AocTypes.BOP),("Novelty", Novelties.N));

In [1]:
deltaFCF.Aggregate().Value

In [1]:
var expectedDeltaFCF_nonFinancial = deltaFCF.Filter(("VariableType", "!IA"), 
                                                    ("VariableType", "!YCU"), 
                                                    ("VariableType", "!CRU")  ).Aggregate().Value;

In [1]:
expectedDeltaFCF_nonFinancial

For consistency, this term is taken with the minus sign

In [1]:
expectedDeltaFCF_nonFinancial.CheckEquality( -computedDeltaFCF_nonFinancial ).Should().Be(true);

### Financial

Conversely, the computed value for the financial contribution to the Fulfillment Cashflows (FCF) is

In [1]:
var computedDeltaFCF_Financial = Test.GetScopes<FcfChangeInEstimate>(identities).Aggregate()
                                     .FpFinancial.Filter(("GroupOfContract", dataNode)).Aggregate().Value;

In [1]:
computedDeltaFCF_Financial

The independent recalculation of this value reads

In [1]:
var expectedDeltaFCF_Financial = (deltaFCF.Filter(("VariableType", "IA")) +
                                  deltaFCF.Filter(("VariableType", "YCU")) +
                                  deltaFCF.Filter(("VariableType", "CRU")) ).Aggregate().Value;

In [1]:
expectedDeltaFCF_Financial

taking the minus sign into account

In [1]:
expectedDeltaFCF_Financial.CheckEquality( -computedDeltaFCF_Financial ).Should().Be(true);

### Other Comprehensive Income

The Other Comprehensive Income (OCI) term is by definition the sum of the financial contributions not related to the insurance sector.

The <code>Default</code> method to compute the OCI term is by subtracting the FCF computed with the *Locked-In* rates and the FCF computed with the *Current* rates. In this way the non insurance financial volatiliy of the economic input is relegated to the OCI. 

In [1]:
var computedDeltaFCF_OCI = Test.GetScopes<FcfChangeInEstimate>(identities).Aggregate()
                               .OciFinancial.Filter(("GroupOfContract", dataNode)).Aggregate().Value;

In [1]:
computedDeltaFCF_OCI

The OCI contribution is zero when the OCI option is disabled for the [target Group of Contract](#report).

Here below we follow the steps to calculate independently the OCI contribution 

In [1]:
var DeltaFCF_Locked = deltaFCF.Aggregate().Value;

In [1]:
var FCFs_current = Test.GetScopes<CurrentFcf>(identities).Aggregate().CurrentFcf.Filter(("GroupOfContract", dataNode));

In [1]:
var DeltaFCF_Current = FCFs_current.Filter(("VariableType", "!BOP"),("VariableType", "!EOP")) +
                       FCFs_current.Filter(("VariableType", AocTypes.BOP),("Novelty", Novelties.N));

In [1]:
var expectedDeltaFCF_OCI = DeltaFCF_Locked - DeltaFCF_Current.Aggregate().Value;

In [1]:
DeltaFCF_Locked

In [1]:
DeltaFCF_Current.Aggregate().Value

In [1]:
expectedDeltaFCF_OCI

In [1]:
expectedDeltaFCF_OCI.CheckEquality( computedDeltaFCF_OCI ).Should().Be(true);

## Contractual Service Margin

### Non Financial

The computed value for the non financial change of the CSM is

In [1]:
var computedDeltaCSM_nonFinancial = Test.GetScopes<CsmChangeInEstimate>(identities).Aggregate()
                                        .NonFinancialChanges.Filter(("GroupOfContract", dataNode)).Aggregate().Value;

In [1]:
computedDeltaCSM_nonFinancial

This value can be independently computed starting from the full CSM data from which the delta can be computed:

In [1]:
var CSM = Test.GetScopes<Csm>(identities).Aggregate().Csm.Filter(("GroupOfContract", dataNode));

In [1]:
var deltaCSM = CSM.Filter(("VariableType", "!BOP"),("VariableType", "!EOP")) +
               CSM.Filter(("VariableType", AocTypes.BOP),("Novelty", Novelties.N));

In [1]:
deltaCSM.Aggregate().Value

The non Financial contribution is calculated subtracting the Amortization and the Interest Accretion terms:

In [1]:
var expectedDeltaCSM_nonFinancial = deltaCSM.Filter(("VariableType", "!AM"), ("VariableType", "!IA"), ("VariableType", "!YCU"), ("VariableType", "!CRU")).Aggregate().Value;

In [1]:
expectedDeltaCSM_nonFinancial

In [1]:
expectedDeltaCSM_nonFinancial.CheckEquality( -computedDeltaCSM_nonFinancial ).Should().Be(true);

### Financial

Conversely, the computed value for the financial change of the CSM is

In [1]:
var computedDeltaCSM_Financial = Test.GetScopes<CsmChangeInEstimate>(identities).Aggregate()
                                     .FinancialChanges.Filter(("GroupOfContract", dataNode)).Aggregate().Value;

In [1]:
computedDeltaCSM_Financial

This value can be independently computed starting from the $\Delta\text{CSM}$, and considering only the Interest Accretion term

In [1]:
var expectedDeltaCSM_Financial = ( deltaCSM.Filter(("VariableType", "IA")) +
                                   deltaCSM.Filter(("VariableType", "YCU")) +
                                   deltaCSM.Filter(("VariableType", "CRU")) )
                                 .Aggregate().Value;

In [1]:
expectedDeltaCSM_Financial

In [1]:
expectedDeltaCSM_Financial.CheckEquality( -computedDeltaCSM_Financial ).Should().Be(true);

## Loss Component

### Non Financial

The computed value for the non financial change of the LC is

In [1]:
var computedDeltaLC_nonFinancial = Test.GetScopes<LcChangeInEstimate>(identities).Aggregate()
                                       .NonFinancialChanges.Filter(("GroupOfContract", dataNode)).Aggregate().Value;

In [1]:
computedDeltaLC_nonFinancial

This value can be independently computed starting from the full CSM data from which the delta can be computed:

In [1]:
var LC = Test.GetScopes<Lc>(identities).Aggregate().Lc.Filter(("GroupOfContract", dataNode));

In [1]:
var deltaLC = LC.Filter(("VariableType", "!BOP"),("VariableType", "!EOP")) +
              LC.Filter(("VariableType", AocTypes.BOP),("Novelty", Novelties.N));

In [1]:
deltaLC.Aggregate().Value

The non Financial contribution is calculated subtracting the Amortization and the Interest Accretion terms:

In [1]:
var expectedDeltaLC_nonFinancial = deltaLC.Filter(("VariableType", "!AM"), ("VariableType", "!IA"), ("VariableType", "!YCU"), ("VariableType", "!CRU")).Aggregate().Value;

In [1]:
expectedDeltaLC_nonFinancial

In [1]:
expectedDeltaLC_nonFinancial.CheckEquality( -computedDeltaLC_nonFinancial ).Should().Be(true);

### Financial

Conversely, the computed value for the financial change of the LC is

In [1]:
var computedDeltaLC_Financial = Test.GetScopes<LcChangeInEstimate>(identities).Aggregate()
                                    .FinancialChanges.Filter(("GroupOfContract", dataNode)).Aggregate().Value;

In [1]:
computedDeltaLC_Financial

This value can be independently computed starting from the $\Delta\text{CSM}$, and considering only the Interest Accretion term

In [1]:
var expectedDeltaLC_Financial = ( deltaLC.Filter(("VariableType", "IA")) +
                                  deltaLC.Filter(("VariableType", "YCU")) +
                                  deltaLC.Filter(("VariableType", "CRU")) )
                                .Aggregate().Value;

In [1]:
expectedDeltaLC_Financial

In [1]:
expectedDeltaLC_Financial.CheckEquality( -computedDeltaLC_Financial ).Should().Be(true);

## Loss Recovery Component

### Non Financial

If the [target Group of Contract](#report) is not of type Re-Insurance, the result of the following will return <code>null</code>.

The computed value for the non financial change of the LoReCo is

In [1]:
var computedDeltaLORECO_nonFinancial = Test.GetScopes<LorecoChangeInEstimate>(identities).Aggregate()
                                           .NonFinancialChanges.Filter(("GroupOfContract", dataNode)).Aggregate()?.Value?? 0;

In [1]:
computedDeltaLORECO_nonFinancial

This value can be independently computed starting from the full LoReCo data from which the delta can be computed:

In [1]:
var LORECO = Test.GetScopes<Loreco>(identities).Aggregate().Loreco.Filter(("GroupOfContract", dataNode));

In [1]:
var deltaLORECO = LORECO.Filter(("VariableType", "!BOP"),("VariableType", "!EOP")) +
                  LORECO.Filter(("VariableType", AocTypes.BOP),("Novelty", Novelties.N));

In [1]:
deltaLORECO.Aggregate()?.Value?? 0

The non Financial contribution is calculated subtracting the Amortization and the Interest Accretion terms:

In [1]:
var expectedDeltaLORECO_nonFinancial = deltaLORECO.Filter(("VariableType", "!AM"), ("VariableType", "!IA"), ("VariableType", "!YCU"), ("VariableType", "!CRU")).Aggregate()?.Value?? 0;

In [1]:
expectedDeltaLORECO_nonFinancial

In [1]:
expectedDeltaLORECO_nonFinancial.CheckEquality( -computedDeltaLORECO_nonFinancial ).Should().Be(true);

### Financial

Conversely, the computed value for the financial change of the LoReCo is

In [1]:
var computedDeltaLORECO_Financial = Test.GetScopes<LorecoChangeInEstimate>(identities).Aggregate()
                                    .FinancialChanges.Filter(("GroupOfContract", dataNode)).Aggregate()?.Value?? 0;

In [1]:
computedDeltaLORECO_Financial

This value can be independently computed starting from the $\Delta\text{CSM}$, and considering only the Interest Accretion term

In [1]:
var expectedDeltaLORECO_Financial = ( deltaLORECO.Filter(("VariableType", "IA")) +
                                     deltaLORECO.Filter(("VariableType", "YCU")) +
                                     deltaLORECO.Filter(("VariableType", "CRU")) )
                                    .Aggregate()?.Value?? 0;

In [1]:
expectedDeltaLORECO_Financial

In [1]:
expectedDeltaLORECO_Financial.CheckEquality( -computedDeltaLORECO_Financial ).Should().Be(true);

<br/><br/>
<a name='reconciliation'></a>
# Reconciliation

Based on the results calculated in Section 3, we check here the consistency of the [Financial Performance Report](#report). 

Finer granular reconciliations can be performed by following the datails of how each report contributes to the Financial Performance Report. Please, refer to the [Report documentation](../Report/ReportScopes#financial-performance) for all the details. 

## Insurance Revenue

The Insurance Revenue (IR) contributions vary from Insurance to Re-Insurance and depend on the Liability Type of the [target Group of Contract](#report). This can be summarized by the formulas below

$$
\text{IR} = \left\{ 
\begin{array}{ll}
\Delta\text{FCF Gross Non Financial} + \Delta\text{CSM Non Financial} + \Delta\text{CSM Release} \\
+ \text{Incurred Incoming Cashflows} + \text{Claims ICO} + \text{Incurred Deferrals} \\
+ \text{Exc. Experience Adjustment on Premiums} ~~,
& \text{for LRC Insurance contracts} \\[0.2cm]
\Delta\text{CSM Non Financial} + \Delta\text{CSM Release} \\
+ \text{Incurred Incoming Cashflows} + \text{Claims ICO} + \text{Incurred Deferrals} ~~,
& \text{for LIC Insurance contracts} \\[0.2cm]
\text{Claims ICO} + \text{Incurred Deferrals} ~~,
& \text{for Re-Insurance contracts}
\end{array}
\right.
$$

Here below we recompute the IR term:

In [1]:
var expectedDeltaCSM_release = deltaCSM.Filter(("VariableType", "AM")).Aggregate()?.Value?? 0;

In [1]:
var expectedIncurredIncomingCashflows = Test.GetScopes<IncurredActuals>(identities).Aggregate().Premiums
                                            .Filter(("GroupOfContract", dataNode)).Aggregate()?.Value?? 0;

In [1]:
var expectedIncurredDeferralsIr = Test.GetScopes<IncurredDeferrals>(identities).Aggregate().AmortizationToIr
                                      .Filter(("GroupOfContract", dataNode)).Aggregate()?.Value?? 0;

In [1]:
var expectedClaimsICO = Test.GetScopes<IncurredActuals>(identities).Aggregate().ClaimsIcoToIr
                            .Filter(("GroupOfContract", dataNode)).Aggregate()?.Value?? 0;

In [1]:
var expectedExcExperienceAdjPremiums = Test.GetScopes<ExperienceAdjustmentOnPremium>(identities).Aggregate().ExperienceAdjustmentOnPremium
                                           .Filter(("GroupOfContract", dataNode)).Aggregate()?.Value?? 0;

In [1]:
double IR = 0;
if(groupOfContract.Partner == null && groupOfContract.LiabilityType == "LRC")
    IR = - expectedDeltaFCF_nonFinancial 
         - expectedDeltaCSM_nonFinancial 
         - expectedDeltaCSM_release 
         + expectedIncurredIncomingCashflows
         + expectedClaimsICO
         + expectedIncurredDeferralsIr
         + expectedExcExperienceAdjPremiums;
else if(groupOfContract.Partner == null && groupOfContract.LiabilityType == "LIC")
    IR = - expectedDeltaCSM_nonFinancial
         - expectedDeltaCSM_release
         + expectedIncurredIncomingCashflows
         + expectedClaimsICO
         + expectedIncurredDeferralsIr 
         + expectedExcExperienceAdjPremiums;
else if(groupOfContract.Partner != null)
    IR = + expectedClaimsICO
         + expectedIncurredDeferralsIr
         + expectedExcExperienceAdjPremiums;

In [1]:
IR

## Insurance Service Expenses

Analogously, the Insurance Service Expenses (ISE) read

$$
\text{ISE} = \left\{ 
\begin{array}{ll}
\Delta\text{LC Non Financial}
+ \Delta\text{LC Release}
+ \text{Incurred ClaimsNIC} \\
+ \text{Incurred Expenses}
+ \text{Incurred Commissions}
+ \text{Incurred Deferrals}
& \text{for LRC Insurance contracts} \\[0.2cm]
\Delta\text{FCF Reinsurance Non Financial}
+ \Delta\text{LC Non Financial}
+ \Delta\text{LC Release}
+ \text{Incurred ClaimsNIC} \\
+ \text{Incurred Expenses} 
+ \text{Incurred Commissions}
+ \text{Incurred Deferrals}
& \text{for LIC Insurance contracts} \\[0.2cm]
\Delta\text{FCF Reinsurance Non Financial} 
+ \Delta\text{CSM Non Financial} 
+ \Delta\text{CSM Release} \\
+ \Delta\text{LC Non Financial} 
+ \Delta\text{LC Release} 
+ \Delta\text{LoReCo Non Financial} \\
+ \text{Incurred Incoming Cashflows} 
+ \text{Incurred ClaimsNIC} 
+ \text{Incurred Expenses} \\
+ \text{Incurred Commissions}
+ \text{Incurred Deferrals}
& \text{for Re-Insurance contracts}
\end{array}
\right.
$$

Here below we recompute the ISE term:

In [1]:
var expectedDeltaLC_release = deltaLC.Filter(("VariableType", "AM")).Aggregate()?.Value?? 0;

In [1]:
var expectedDeltaLoReCo_release = deltaLORECO.Filter(("VariableType", "AM")).Aggregate()?.Value?? 0;

In [1]:
var expectedIncurredClaimsNIC = Test.GetScopes<IncurredActuals>(identities).Aggregate().ClaimsNic
                                    .Filter(("GroupOfContract", dataNode)).Aggregate()?.Value?? 0;

In [1]:
var expectedIncurredExpenses = Test.GetScopes<IncurredActuals>(identities).Aggregate().Expenses
                                   .Filter(("GroupOfContract", dataNode)).Aggregate()?.Value?? 0;

In [1]:
var expectedIncurredCommissions = Test.GetScopes<IncurredActuals>(identities).Aggregate().Commissions
                                      .Filter(("GroupOfContract", dataNode)).Aggregate()?.Value?? 0;

In [1]:
var expectedIncurredClaimsICO = Test.GetScopes<IncurredActuals>(identities).Aggregate().ClaimsIcoToIse
                            .Filter(("GroupOfContract", dataNode)).Aggregate()?.Value?? 0;

In [1]:
var expectedIncurredDeferralsIse = Test.GetScopes<IncurredDeferrals>(identities).Aggregate().AmortizationToIse
                                       .Filter(("GroupOfContract", dataNode)).Aggregate()?.Value?? 0;

In [1]:
double ISE = 0;
if(groupOfContract.Partner == null && groupOfContract.LiabilityType == "LRC")
    ISE = - expectedDeltaLC_nonFinancial 
          - expectedDeltaLC_release
          + expectedIncurredClaimsNIC 
          + expectedIncurredExpenses 
          + expectedIncurredClaimsICO
          + expectedIncurredCommissions
          + expectedIncurredDeferralsIse;
else if(groupOfContract.Partner == null && groupOfContract.LiabilityType == "LIC")
    ISE = - expectedDeltaFCF_nonFinancial
          - expectedDeltaLC_nonFinancial 
          - expectedDeltaLC_release
          + expectedIncurredClaimsNIC 
          + expectedIncurredExpenses 
          + expectedIncurredClaimsICO
          + expectedIncurredCommissions
          + expectedIncurredDeferralsIse;
else if(groupOfContract.Partner != null)
    ISE = - expectedDeltaFCF_nonFinancial
          - expectedDeltaCSM_nonFinancial
          - expectedDeltaCSM_release
          - expectedDeltaLC_nonFinancial 
          - expectedDeltaLC_release
          - expectedDeltaLoReCo_release
          + expectedIncurredIncomingCashflows
          + expectedIncurredClaimsNIC 
          + expectedIncurredExpenses 
          + expectedIncurredClaimsICO
          + expectedIncurredCommissions
          + expectedIncurredDeferralsIse;

In [1]:
ISE

## Insurance Finance Income/Expenses

The Financial part associated to the Insurance business encompassing both the Income and Expenses is denoted IFIE from the initials marked in capital letters. The  

$$
\text{IFIE} = \left\{
\begin{array}{ll}
\Delta\text{FCF Financial} + \Delta\text{CSM Financial} + \Delta\text{LC Financial} ~~,
& \text{for Insurance contracts} \\[0.2cm]
\Delta\text{FCF Financial} + \Delta\text{CSM Financial} + \Delta\text{LC Financial} 
+ \Delta\text{LoReCo Financial} ~~, 
& \text{for Re-Insurance contracts}
\end{array}
\right.
$$

Here below we recompute the IFIE term:

In [1]:
double IFIE = 0;
if(groupOfContract.Partner == null) 
    IFIE = - expectedDeltaFCF_Financial 
           - expectedDeltaCSM_Financial
           - expectedDeltaLC_Financial;
else
    IFIE = - expectedDeltaFCF_Financial 
           - expectedDeltaCSM_Financial
           - expectedDeltaLC_Financial
           - expectedDeltaLORECO_Financial;

In [1]:
IFIE

## Other Comprehensive Income

The Other Comprehensive Income (OCI) can be non-zero only when the OCI is enabled for [target Group of Contract](#report), that is, <code>Oci Type = Default</code>.

$$
\text{OCI} = \Delta\text{FCF} \big|_{\text{Locked-in}} - \Delta\text{FCF} \big|_{\text{Current}} ~~.
$$

Here below we recompute the OCI term:

In [1]:
var OCI = expectedDeltaFCF_OCI;

In [1]:
OCI